In [151]:
import requests

def get_prices(symbol, key, date_param='5d'):

    url_prefix = "https://cloud.iexapis.com/stable/"

    path = f'stock/{symbol}/chart/{date_param}?chartCloseOnly=True&&token={key}'
    print(f"Fetching {date_param} data for {symbol}")
    full_url = requests.compat.urljoin(url_prefix, path)

    try:
        resp = requests.get(full_url)
    except Exception as e:
        print(f"Exception {e} occurred!")
        return None

    if resp.status_code != 200:
        print(f"Uh oh, something's wrong! Response code {resp.status_code} received.")
        return resp

    else:
        print(f"Got the data")
        return resp

In [152]:
import os
import json

def get_cached_prices(symbol, date_range, outpath):
    with open("./tokens/iex_token.txt", "r") as f:
        iex_tkn = f.read().strip()

    if not os.path.exists(outpath):
        resp = get_prices(symbol, iex_tkn, date_param=date_range)
        if resp is not None:
            prices_obj = json.loads(resp.text)
            with open(outpath, "w") as f:
                json.dump(prices_obj, f)
            return prices_obj
    else:
        with open(outpath, "r") as f:
            data_obj = json.load(f)
        return data_obj

In [153]:
symbols = ["MSFT", "AAPL", "NVDA", "JNJ", "KHC", "ALL"]

In [154]:
def getDf(symbol, data):
    df = pd.DataFrame(data)
    df = df[['date', 'close']]
    df['date'] = pd.to_datetime(df.date)
    df.columns = ['date', symbol]
    df.set_index('date')
    return df;

In [155]:
import pandas as pd

date_range = "5y"

symbols_dict = dict()

for symbol in symbols:
    outpath = f"./.data/stocks/{symbol}_{date_range}.json"
    symbols_dict[symbol] = get_cached_prices(symbol, date_range, outpath)

Fetching 5y data for MSFT
Got the data
Fetching 5y data for AAPL
Got the data
Fetching 5y data for NVDA
Got the data
Fetching 5y data for JNJ
Got the data
Fetching 5y data for KHC
Got the data
Fetching 5y data for ALL
Got the data


In [156]:
import utils

df = utils.symbol_dict_to_df(symbols_dict)
df['date'] = pd.to_datetime(df.date)
df = df.pivot_table(values='close', index='date', columns='symbol', aggfunc='last')
df

symbol,AAPL,ALL,JNJ,KHC,MSFT,NVDA
date,,,,,,
2017-03-03,34.945,81.86,123.79,91.50,64.25,24.6075
2017-03-06,34.835,81.34,123.71,91.16,64.27,24.4175
2017-03-07,34.880,81.46,123.83,90.80,64.40,24.6850
2017-03-08,34.750,81.13,124.10,90.90,64.99,24.6400
2017-03-09,34.670,81.57,125.95,91.53,64.73,24.6350
...,...,...,...,...,...,...
2022-02-23,160.070,123.28,161.22,40.10,280.27,223.8700
2022-02-24,162.740,120.82,158.14,38.98,294.59,237.4800
2022-02-25,164.850,124.17,166.00,39.73,297.31,241.5700


In [159]:
from pypfopt import EfficientFrontier, risk_models, expected_returns, plotting

mu = expected_returns.mean_historical_return(df, frequency=252)

In [161]:
S = risk_models.sample_cov(df, frequency=252)

symbol,AAPL,ALL,JNJ,KHC,MSFT,NVDA
symbol,,,,,,
AAPL,0.094714,0.033268,0.025207,0.032901,0.064083,0.089718
ALL,0.033268,0.064595,0.024420,0.033749,0.032841,0.035772
JNJ,0.025207,0.024420,0.041190,0.024585,0.027249,0.028000
KHC,0.032901,0.033749,0.024585,0.097620,0.030670,0.035120
MSFT,0.064083,0.032841,0.027249,0.030670,0.078638,0.089727
NVDA,0.089718,0.035772,0.028000,0.035120,0.089727,0.228820


In [173]:
ef = EfficientFrontier(mu, S)

In [174]:
weights = ef.min_volatility()

In [175]:
weights

OrderedDict([('AAPL', 0.060617254780653),
             ('ALL', 0.1977101574069492),
             ('JNJ', 0.577960004405767),
             ('KHC', 0.0966182132113338),
             ('MSFT', 0.067094370195297),
             ('NVDA', 0.0)])

In [176]:
ef.portfolio_performance(verbose=True)

Expected annual return: 8.0%
Annual volatility: 18.5%
Sharpe Ratio: 0.32


(0.07983792450704413, 0.18538001979798974, 0.322785187811772)